### Import required libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from CartPole import CartPole
from CartPole_GPS import CartPole_GPS

from ilqr.dynamics import constrain
from copy import deepcopy

from EstimateDynamics import local_estimate
from GMM import Estimated_Dynamics_Prior

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

from mujoco_py import load_model_from_path, MjSim, MjViewer
import mujoco_py

import time

### Formulate the iLQR problem

In [2]:
'''
1 - dt = time step
2 - N = Number of control points in the trajectory
3 - x0 = Initial state
4 - x_goal = Final state
5 - Q = State cost
6 - R = Control cost
7 - Q_terminal = Cost at the final step
8 - x_dynamics array stores the information regarding system. 
    x_dynamics[0] = m = mass of the pendulum bob 
    x_dynamics[1] = M = mass of the cart 
    x_dynamics[2] = L = length of the massles|s rod 
    x_dynamics[3] = g = gravity 
    x_dynamics[4] = d = damping in the system
'''
dt = 0.005
N = 2500  # Number of time steps in trajectory.
x_dynamics = np.array([1, 5, 2, 9.80665, 0.1]) # m=1, M=5, L=0.3, g=9.80665, d=0.05
x0 = np.array([-2.0, 0.0, 0.01, 0.0])  # Initial state
x_goal = np.array([3.0, 0.0, 0.0, 0.0])
# Instantenous state cost.
Q = np.eye(5)
# Q[0,0] = 10
Q[2, 2] = 10
Q[3, 3] = 10
# Q[4, 4] = 100
# Terminal state cost.
Q_terminal = 100 * np.eye(5)
# Instantaneous control cost.
R = np.array([[1.0]])

### iLQR on Cart Pole

In [4]:
cartpole_prob = CartPole(dt, N, x_dynamics, x0, x_goal, Q, R, Q_terminal)
xs, us, K, k = cartpole_prob.run_IterLinQuadReg()

iteration 0 accepted 66581.56537599936 [ 3.75171071  0.33857974 -0.96274072 -1.71126115]
iteration 1 failed 66581.56537599934 [ 3.75171071  0.33857974 -0.96274072 -1.71126115]
iteration 2 accepted 61593.54284321188 [3.01092651 0.16619785 1.40182441 2.59581977]
iteration 3 accepted 58403.52980858578 [ 3.62655133  0.31541928 -0.24913086 -0.40402417]
iteration 4 accepted 58277.26022968569 [ 3.6771218   0.27843403 -0.18930162 -0.27932402]
iteration 5 accepted 58076.51720890989 [ 3.66501513  0.19578397 -0.08222649 -0.0599065 ]
iteration 6 accepted 57784.08022350561 [ 3.31461579 -0.12721779  0.35449206  0.82162713]
iteration 7 accepted 57703.36301419336 [ 3.44803114 -0.11186697  0.49231306  1.09957958]
iteration 8 accepted 57698.7403594128 [ 3.62902902 -0.11147383  0.52706734  1.15881444]
iteration 9 accepted 57468.9232502335 [ 3.37654544 -0.08872058  0.28941388  0.69994074]
iteration 10 accepted 57468.76680512249 [ 3.37679196 -0.08855749  0.28914052  0.69939193]
iteration 11 accepted 57468.

iteration 92 accepted 56074.43758032304 [ 3.42222738 -0.171069   -0.03169584  0.84634009]
iteration 93 accepted 56068.09426664553 [ 3.43242094 -0.17489953 -0.0407564   0.85425249]
iteration 94 accepted 56062.30152653152 [ 3.42917109 -0.17642304 -0.06359468  0.85379009]
iteration 95 accepted 56060.74985599138 [ 3.45003191 -0.18133026 -0.05465109  0.8760604 ]
iteration 96 accepted 56055.09713247999 [ 3.44280129 -0.18431026 -0.07729047  0.86587749]
iteration 97 accepted 56043.98671257333 [ 3.44544147 -0.18436261 -0.0896009   0.88165644]
iteration 98 accepted 56043.06164042159 [ 3.46258526 -0.18407012 -0.04434314  0.87703425]
iteration 99 accepted 56033.87284327714 [ 3.43276283 -0.18291555 -0.07873216  0.87684482]


In [ ]:
# State matrix split into individual states. For plotting and analysing purposes.
t = np.arange(N + 1) * dt
x = xs[:, 0] # Position
x_dot = xs[:, 1] # Velocity
theta = np.unwrap(cartpole_prob.deaugment_state(xs)[:, 2])  # Theta, makes for smoother plots.
theta_dot = xs[:, 3] # Angular velocity

### Simulate the real system and generate the data
Cost matrices, initial position and goal position will remain same as the above problem. As it indicates one policy. But still the initial positions and goal positions must be passed explicitly to the function. But you don't need to pass cost matrices (assume penalty on the system is same), this is just used to use to calculate the cost of the trajectory. Correct control action must be passed. Parameter gamma indicates how much of original data you want to keep

Variance of the Gaussian noise will be taken as input from a Unif(0, var_range) uniform distribution. Inputs: x_initial, x_goal, u, n_rollouts, pattern='Normal', pattern_rand=False, var_range=10, gamma=0.2, percent=20

Pattern controls how the control sequence will be modified after applying white Guassian noise (zero mean).
- Normal: based on the correction/mixing parameter gamma generate control (gamma controls how much noise we want).
- MissingValue: based on the given percentage, set those many values to zero (it is implicitly it uses "Normal" generated control is used). 
- Shuffle: shuffles the entire "Normal" generated control sequence.
- TimeDelay: takes the "Normal" generated control and shifts it by 1 index i.e. one unit time delay.
- Extreme: sets gamma as zeros and generates control based on only noise.

If 'pattern_rand' is 'True' then we don't need to send the explicitly, it will chose one randomly for every rollout (default is 'False'). If you want to chose specific pattern then send it explicitly. 

In [ ]:
x_rollout, u_rollout, local_policy, cost = cartpole_prob.gen_rollouts(x0, x_goal, us, n_rollouts=10, pattern_rand=True, var_range=10, gamma=0.2, percent=20)

### Local system dynamics/model estimate
loca_estimate: function takes the states (arranged in a special format, [x(t), u(t), x(t+1)]), no. of gaussian mixtures and no.of states.

In [ ]:
model = Estimated_Dynamics_Prior(init_sequential=False, eigreg=False, warmstart=True, 
                 min_samples_per_cluster=20, max_clusters=50, max_samples=20, strength=1.0)
model.update_prior(x_rollout, u_rollout)
A, B, C = model.fit(x_rollout, u_rollout)

In [ ]:
print(A.shape)
print(B.shape)
print(C.shape)

In [10]:
Model = "mujoco/cartpole.xml"
model_loaded = load_model_from_path(Model)
sim = MjSim(model_loaded)

In [11]:
#viewer = mujoco_py.MjViewer(sim)
t = 0
sim.data.qpos[0] = 0.0
sim.data.qpos[1] = 0.01
sim.data.qvel[0] = 0
sim.data.qvel[1] = 0
final = 0
for i in range(1500):
    start_time = time.time()
    state = np.c_[sim.data.qpos[0],sim.data.qvel[0],np.sin(sim.data.qpos[1]),
                  np.cos(sim.data.qpos[1]),sim.data.qvel[1]].T
    control = np.dot(k[i,:],(state - xs[i].reshape(5,1) ))  + K[i].T + us[i]
    sim.data.ctrl[0] = control
    sim.step()
    #viewer.render()
    #print(control, "this is the control")
    if (sim.data.qpos[0] == 1.0 and sim.data.qpos[1] == 0):
        print('states reached')
        break
print(sim.get_state())

MjSimState(time=7.499999999999862, qpos=array([0.95906096, 2.94081605]), qvel=array([0.07484495, 3.28427469]), act=None, udd_state={})


In [14]:
import time
time.sleep(5)

In [15]:
from Simulator import Mujoco_sim
cart_pole_simulator = Mujoco_sim(Model,True)
cart_pole_simulator.load(xs,us,k,K,x0,initial=False)

model added to the simulator


In [16]:
cart_pole_simulator.runSimulation()

Creating window glfw
